# Midterm Project - wnz7kd - Chinook Database

## Project Overview
The purpose of this project is to build a Data Warehouse by integrating multiple data sources, including SQL, NoSQL, CSV, and JSON formats. This warehouse aims to support business analytics by centralizing data and enabling structured insights, such as identifying top spenders and genre popularity.

## Objective
This data warehouse allows for comprehensive analysis of sales data, customer behaviors, and product performance. By building dimensional tables and a sales fact table, we can support in-depth querying and reporting capabilities that can help businesses make informed decisions.

## Data Sources
This project integrates data from multiple sources:

- **SQL Database (Chinook OLTP)**: Stores primary transactional data, such as products (tracks), invoices, and invoice lines.
- **MongoDB (JSON Data)**: Contains invoice data stored in JSON format, loaded into MongoDB.
- **CSV File**: Stores customer information for additional dimensional data in the warehouse.
- **JSON File**: Stores nested data that is parsed and loaded into MongoDB for use in the warehouse.

Each source serves as input for a specific dimension or fact table in the final data warehouse, providing a rich view of business information.

### Project Setup and Configuration

In [1]:
import os
import json
import numpy
import datetime
import certifi
import pandas as pd
from sqlalchemy import create_engine, text

import pymongo
import sqlalchemy

In [2]:
#define connection parameters
mysql_args = {
    "uid": "root",
    "pwd": "CPayne091203",
    "hostname": "localhost",
    "dbname": "chinook"
}
#set up connection ot MongoDB
mongodb_args = {
    "user_name": "cpayne",
    "password": "CPayne57522",
    "cluster_name": "sandbox",
    "cluster_subnet": "pn3ju",
    "cluster_location": "atlas",      
    "db_name": "chinook"
}

In [3]:
def get_sql_dataframe(sql_query, **args):
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection)
    connection.close()
    return dframe
    
def set_dataframe(df, table_name, pk_column, db_operation, **args):
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});"))
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    connection.close()

def get_mongo_client(**args):
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    if args["cluster_location"] == "atlas":
        connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
        client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
    else:
        client = pymongo.MongoClient("mongodb://localhost:27017/")
    return client

def get_mongo_dataframe(mongo_client, db_name, collection, query):
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    # Check if '_id' column exists before attempting to drop it
    if '_id' in dframe.columns:
        dframe.drop(['_id'], axis=1, inplace=True)
    
    mongo_client.close()
    
    return dframe

def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
    mongo_client.close()

def set_mongo_collections_from_nested_json(mongo_client, db_name, json_file_path):
    db = mongo_client[db_name]

    # Load the nested JSON structure
    with open(json_file_path, 'r') as openfile:
        json_data = json.load(openfile)

    # Iterate over each key in the JSON file (e.g., Genre, MediaType)
    for collection_name, documents in json_data.items():
        if isinstance(documents, list) and documents:
            db[collection_name].drop()  # Drop collection if it exists
            db[collection_name].insert_many(documents)  # Insert documents into MongoDB collection
        else:
            print(f"Skipped {collection_name}: Not a list or empty.")
    
    mongo_client.close()


#### Database Connections and Data Loading
##### Establish connections to OLTP, OLAP databases, MongoDB, and load JSON data if required.

In [8]:
#OLTP database (source)
chinook_db = create_engine(f"mysql+pymysql://{mysql_args['uid']}:{mysql_args['pwd']}@{mysql_args['hostname']}/chinook")
#OLAP DW (target)
data_mart_db = create_engine(f"mysql+pymysql://{mysql_args['uid']}:{mysql_args['pwd']}@{mysql_args['hostname']}/chinook_data_mart")
mongo_client = get_mongo_client(**mongodb_args)
json_file_path = "/Users/charlie/Desktop/DS-2002/Projects/ChinookData.json"  # Update with the actual path to your JSON file


# Check if the collections are already loaded in MongoDB
db = mongo_client[mongodb_args["db_name"]]
if "Invoice" not in db.list_collection_names():
    # Load JSON data into MongoDB if not already loaded
    set_mongo_collections_from_nested_json(mongo_client, mongodb_args["db_name"], json_file_path)
else:
    print("JSON data already loaded in MongoDB.")


JSON data already loaded in MongoDB.



## ETL Pipeline

### Extraction
Data is extracted from SQL (OLTP), MongoDB, and CSV sources. The extraction process includes retrieving necessary fields, which are then transformed and loaded into the data warehouse.

### Transformation
Various transformations are applied, including:
- **Product Duration Calculation**: Duration is formatted as MM:SS from the raw `Milliseconds` field.
- **Data Cleaning**: Checking for null values, duplicates, and ensuring unique keys for dimensional integrity.

### Loading
The transformed data is loaded into the `chinook_data_mart` (OLAP) database. Dimension tables (`Customer_Dim`, `Product_Dim`, `Invoice_Dim`) and the fact table (`Sales_Fact`) are structured to enable efficient querying.



### Data Extraction from Multiple Sources

In [11]:
# Extract data from MySQL (Chinook OLTP) using get_sql_dataframe
product_df = get_sql_dataframe("SELECT TrackId AS ProductID, Name, AlbumId FROM Track", **mysql_args)
print("Product Data from SQL:")
product_df.head()

Product Data from SQL:


,ProductID,Name,AlbumId
0,1,For Those About To Rock (We Salute You),1
1,2,Balls to the Wall,2
2,3,Fast As a Shark,3
3,4,Restless and Wild,3
4,5,Princess of the Dawn,3


In [13]:
# Extract Data from MongoDB
invoice_df = get_mongo_dataframe(mongo_client, mongodb_args["db_name"], "Invoice", {})
print("Invoice Data from MongoDB:")
invoice_df.head()

Invoice Data from MongoDB:


,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,37,17,2021-06-06T00:00:00,1 Microsoft Way,Redmond,WA,USA,98052-8300,3.96
1,6,37,2021-01-19T00:00:00,Berger Straße 10,Frankfurt,,Germany,60316,0.99
2,19,40,2021-03-14T00:00:00,"8, Rue Hanovre",Paris,,France,75002,13.86
3,17,25,2021-03-06T00:00:00,319 N. Frances Street,Madison,WI,USA,53703,5.94
4,10,46,2021-02-03T00:00:00,3 Chatham Street,Dublin,Dublin,Ireland,,5.94


In [15]:
# Extract Data from CSV file
customer_csv_df = pd.read_csv("customer_data.csv")
print("CSV Dimension (Customer):")
customer_csv_df.head()

CSV Dimension (Customer):


,CustomerID,FirstName,LastName,Email,Phone,City,Country,PostalCode,SupportRepId
0,1,Luís,Gonçalves,luisg@embraer.com.br,+55 (12) 3923-5555,São José dos Campos,Brazil,12227-000,3
1,2,Leonie,Köhler,leonekohler@surfeu.de,+49 0711 2842222,Stuttgart,Germany,70174,5
2,3,François,Tremblay,ftremblay@gmail.com,+1 (514) 721-4711,Montréal,Canada,H2G 1A7,3
3,4,Bjørn,Hansen,bjorn.hansen@yahoo.no,+47 22 44 22 22,Oslo,Norway,0171,4
4,5,František,Wichterlová,frantisekw@jetbrains.com,+420 2 4172 5555,Prague,Czech Republic,14700,4


### Data Transformation

#### Product Dimension

In [19]:
product_data_query = """
SELECT 
    Track.TrackId AS ProductID,
    Track.Name AS ProductName,
    Album.Title AS AlbumTitle,
    Artist.Name AS ArtistName,
    Genre.Name AS Genre,
    MediaType.Name AS MediaType,
    Track.Composer AS Composer,
    Track.Milliseconds AS Milliseconds,
    Track.Bytes AS Bytes,
    Track.UnitPrice AS UnitPrice
FROM 
    Track
JOIN 
    Album ON Track.AlbumId = Album.AlbumId
JOIN 
    Artist ON Album.ArtistId = Artist.ArtistId
JOIN 
    Genre ON Track.GenreId = Genre.GenreId
JOIN 
    MediaType ON Track.MediaTypeId = MediaType.MediaTypeId;
"""
product_df = get_sql_dataframe(product_data_query, **mysql_args)

In [21]:
print("Product Dimension Data:")
product_df.head()

Product Dimension Data:


,ProductID,ProductName,AlbumTitle,ArtistName,Genre,MediaType,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),For Those About To Rock We Salute You,AC/DC,Rock,MPEG audio file,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,6,Put The Finger On You,For Those About To Rock We Salute You,AC/DC,Rock,MPEG audio file,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99
2,7,Let's Get It Up,For Those About To Rock We Salute You,AC/DC,Rock,MPEG audio file,"Angus Young, Malcolm Young, Brian Johnson",233926,7636561,0.99
3,8,Inject The Venom,For Those About To Rock We Salute You,AC/DC,Rock,MPEG audio file,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.99
4,9,Snowballed,For Those About To Rock We Salute You,AC/DC,Rock,MPEG audio file,"Angus Young, Malcolm Young, Brian Johnson",203102,6599424,0.99


In [23]:
# Calculate minutes and seconds separately
product_df['Minutes'] = product_df['Milliseconds'] // 60000  # Get full minutes
product_df['Seconds'] = (product_df['Milliseconds'] % 60000) // 1000  # Get remaining seconds

# Create a formatted time column as 'MM:SS'
product_df['Duration'] = product_df['Minutes'].astype(str).str.zfill(2) + ":" + product_df['Seconds'].astype(str).str.zfill(2)

# Drop the original Milliseconds, Minutes, and Seconds columns if no longer needed
product_df = product_df.drop(columns=['Milliseconds', 'Minutes', 'Seconds'])

# Display the modified DataFrame
print("Product Dimension Data (with Duration in MM:SS format):")
product_df.head()

Product Dimension Data (with Duration in MM:SS format):


,ProductID,ProductName,AlbumTitle,ArtistName,Genre,MediaType,Composer,Bytes,UnitPrice,Duration
0,1,For Those About To Rock (We Salute You),For Those About To Rock We Salute You,AC/DC,Rock,MPEG audio file,"Angus Young, Malcolm Young, Brian Johnson",11170334,0.99,05:43
1,6,Put The Finger On You,For Those About To Rock We Salute You,AC/DC,Rock,MPEG audio file,"Angus Young, Malcolm Young, Brian Johnson",6713451,0.99,03:25
2,7,Let's Get It Up,For Those About To Rock We Salute You,AC/DC,Rock,MPEG audio file,"Angus Young, Malcolm Young, Brian Johnson",7636561,0.99,03:53
3,8,Inject The Venom,For Those About To Rock We Salute You,AC/DC,Rock,MPEG audio file,"Angus Young, Malcolm Young, Brian Johnson",6852860,0.99,03:30
4,9,Snowballed,For Those About To Rock We Salute You,AC/DC,Rock,MPEG audio file,"Angus Young, Malcolm Young, Brian Johnson",6599424,0.99,03:23


In [25]:
print("Null values in Product Data:")
print(product_df.isnull().sum())

Null values in Product Data:
ProductID        0
ProductName      0
AlbumTitle       0
ArtistName       0
Genre            0
MediaType        0
Composer       977
Bytes            0
UnitPrice        0
Duration         0
dtype: int64


In [27]:
print("Unique Product IDs:", product_df['ProductID'].is_unique)

Unique Product IDs: True


In [29]:
#Switch to OLAP `chinook_data_mart` for loading
mysql_args["dbname"] = "chinook_data_mart"
# Load the Product_Dim table into the OLAP database
set_dataframe(product_df, "Product_Dim", "ProductID", "insert", **mysql_args)

In [31]:
# Verify tables in chinook_data_mart to confirm Product_Dim exists
tables_in_data_mart = get_sql_dataframe("SHOW TABLES", **mysql_args)
print("Tables in chinook_data_mart:")
print(tables_in_data_mart)

# Query Product_Dim to verify data in the OLAP database
product_dim_sample = get_sql_dataframe("SELECT * FROM Product_Dim LIMIT 5", **mysql_args)
print("Sample data from Product_Dim in chinook_data_mart:")
product_dim_sample.head()

Tables in chinook_data_mart:
  Tables_in_chinook_data_mart
0                    dim_date
1                 Product_Dim
Sample data from Product_Dim in chinook_data_mart:


,ProductID,ProductName,AlbumTitle,ArtistName,Genre,MediaType,Composer,Bytes,UnitPrice,Duration
0,1,For Those About To Rock (We Salute You),For Those About To Rock We Salute You,AC/DC,Rock,MPEG audio file,"Angus Young, Malcolm Young, Brian Johnson",11170334,0.99,05:43
1,2,Balls to the Wall,Balls to the Wall,Accept,Rock,Protected AAC audio file,"U. Dirkschneider, W. Hoffmann, H. Frank, P. Ba...",5510424,0.99,05:42
2,3,Fast As a Shark,Restless and Wild,Accept,Rock,Protected AAC audio file,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",3990994,0.99,03:50
3,4,Restless and Wild,Restless and Wild,Accept,Rock,Protected AAC audio file,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",4331779,0.99,04:12
4,5,Princess of the Dawn,Restless and Wild,Accept,Rock,Protected AAC audio file,Deaffy & R.A. Smith-Diesel,6290521,0.99,06:15


#### Customer Dimension

In [34]:
# Check for missing values
print("Null values in Customer Data:")
print(customer_csv_df.isnull().sum())

# Verify data types
print("Data types in Customer Data:")
print(customer_csv_df.dtypes)

Null values in Customer Data:
CustomerID      0
FirstName       0
LastName        0
Email           0
Phone           1
City            0
Country         0
PostalCode      4
SupportRepId    0
dtype: int64
Data types in Customer Data:
CustomerID       int64
FirstName       object
LastName        object
Email           object
Phone           object
City            object
Country         object
PostalCode      object
SupportRepId     int64
dtype: object


In [36]:
customer_csv_df.head()

,CustomerID,FirstName,LastName,Email,Phone,City,Country,PostalCode,SupportRepId
0,1,Luís,Gonçalves,luisg@embraer.com.br,+55 (12) 3923-5555,São José dos Campos,Brazil,12227-000,3
1,2,Leonie,Köhler,leonekohler@surfeu.de,+49 0711 2842222,Stuttgart,Germany,70174,5
2,3,François,Tremblay,ftremblay@gmail.com,+1 (514) 721-4711,Montréal,Canada,H2G 1A7,3
3,4,Bjørn,Hansen,bjorn.hansen@yahoo.no,+47 22 44 22 22,Oslo,Norway,0171,4
4,5,František,Wichterlová,frantisekw@jetbrains.com,+420 2 4172 5555,Prague,Czech Republic,14700,4


In [38]:
# Loading: Switch to OLAP database and load Customer_Dim into chinook_data_mart
mysql_args["dbname"] = "chinook_data_mart"
set_dataframe(customer_csv_df, "Customer_Dim", "CustomerID", "insert", **mysql_args)

In [40]:
# Verify that Customer_Dim was successfully loaded
customer_dim_sample = get_sql_dataframe("SELECT * FROM Customer_Dim LIMIT 5", **mysql_args)
print("Sample data from Customer_Dim in chinook_data_mart:")
customer_dim_sample.head()

Sample data from Customer_Dim in chinook_data_mart:


,CustomerID,FirstName,LastName,Email,Phone,City,Country,PostalCode,SupportRepId
0,1,Luís,Gonçalves,luisg@embraer.com.br,+55 (12) 3923-5555,São José dos Campos,Brazil,12227-000,3
1,2,Leonie,Köhler,leonekohler@surfeu.de,+49 0711 2842222,Stuttgart,Germany,70174,5
2,3,François,Tremblay,ftremblay@gmail.com,+1 (514) 721-4711,Montréal,Canada,H2G 1A7,3
3,4,Bjørn,Hansen,bjorn.hansen@yahoo.no,+47 22 44 22 22,Oslo,Norway,0171,4
4,5,František,Wichterlová,frantisekw@jetbrains.com,+420 2 4172 5555,Prague,Czech Republic,14700,4


#### Invoice Dimension

In [43]:
invoice_df.head()

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,37,17,2021-06-06T00:00:00,1 Microsoft Way,Redmond,WA,USA,98052-8300,3.96
1,6,37,2021-01-19T00:00:00,Berger Straße 10,Frankfurt,,Germany,60316,0.99
2,19,40,2021-03-14T00:00:00,"8, Rue Hanovre",Paris,,France,75002,13.86
3,17,25,2021-03-06T00:00:00,319 N. Frances Street,Madison,WI,USA,53703,5.94
4,10,46,2021-02-03T00:00:00,3 Chatham Street,Dublin,Dublin,Ireland,,5.94


In [45]:
# Transform Invoice Data for Sales_Fact
invoice_df = invoice_df.rename(columns={
    'InvoiceId': 'InvoiceID',
    'CustomerId': 'CustomerID',
    'InvoiceDate': 'Date',
    'Total': 'TotalAmount'
})
# Convert Date to a datetime format if needed
invoice_df['Date'] = pd.to_datetime(invoice_df['Date'])

In [47]:
print("Transformed Invoice Data:")
invoice_df.head()

Transformed Invoice Data:


,InvoiceID,CustomerID,Date,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,TotalAmount
0,37,17,2021-06-06,1 Microsoft Way,Redmond,WA,USA,98052-8300,3.96
1,6,37,2021-01-19,Berger Straße 10,Frankfurt,,Germany,60316,0.99
2,19,40,2021-03-14,"8, Rue Hanovre",Paris,,France,75002,13.86
3,17,25,2021-03-06,319 N. Frances Street,Madison,WI,USA,53703,5.94
4,10,46,2021-02-03,3 Chatham Street,Dublin,Dublin,Ireland,,5.94


In [49]:
#Load Invoice
mysql_args["dbname"] = "chinook_data_mart"
set_dataframe(invoice_df, "Invoice_Dim", "InvoiceID", "insert", **mysql_args)

In [51]:
invoice_dim_sample = get_sql_dataframe("SELECT * FROM Invoice_Dim LIMIT 5", **mysql_args)
print("Sample data from Invoice_Dim in chinook_data_mart:")
invoice_dim_sample.head()

Sample data from Invoice_Dim in chinook_data_mart:


,InvoiceID,CustomerID,Date,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,TotalAmount
0,1,2,2021-01-01,Theodor-Heuss-Straße 34,Stuttgart,,Germany,70174,1.98
1,2,4,2021-01-02,Ullevålsveien 14,Oslo,,Norway,0171,3.96
2,3,8,2021-01-03,Grétrystraat 63,Brussels,,Belgium,1000,5.94
3,4,14,2021-01-06,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2021-01-11,69 Salem Street,Boston,MA,USA,2113,13.86


#### Invoice Line

In [54]:
mysql_args["dbname"] = "chinook"
invoice_line_df = get_sql_dataframe("SELECT * FROM InvoiceLine", **mysql_args)
print("Invoice Line Data Sample:")
invoice_line_df.head()

Invoice Line Data Sample:


,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity
0,1,1,2,0.99,1
1,2,1,4,0.99,1
2,3,2,6,0.99,1
3,4,2,8,0.99,1
4,5,2,10,0.99,1


In [56]:
invoice_line_df = invoice_line_df.rename(columns={
    'InvoiceLineId': 'InvoiceLineID',
    'InvoiceId': 'InvoiceID',
    'TrackId': 'ProductID',
    'UnitPrice': 'Price'
})
print("Transformed Invoice Line Data:")
invoice_line_df.head()

Transformed Invoice Line Data:


,InvoiceLineID,InvoiceID,ProductID,Price,Quantity
0,1,1,2,0.99,1
1,2,1,4,0.99,1
2,3,2,6,0.99,1
3,4,2,8,0.99,1
4,5,2,10,0.99,1


#### Track data

In [59]:
# Extracting Track Data
track_df = get_sql_dataframe("SELECT TrackID AS ProductID, Name, AlbumID, GenreID, Composer FROM Track", **mysql_args)
print("Track Data Sample:")
track_df.head()

Track Data Sample:


,ProductID,Name,AlbumID,GenreID,Composer
0,1,For Those About To Rock (We Salute You),1,1,"Angus Young, Malcolm Young, Brian Johnson"
1,2,Balls to the Wall,2,1,"U. Dirkschneider, W. Hoffmann, H. Frank, P. Ba..."
2,3,Fast As a Shark,3,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho..."
3,4,Restless and Wild,3,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D..."
4,5,Princess of the Dawn,3,1,Deaffy & R.A. Smith-Diesel


In [61]:
# Optional: Check for null values or duplicates
print("Null values in Track Data:")
print(track_df.isnull().sum())
print("Unique Product IDs:", track_df['ProductID'].is_unique)

Null values in Track Data:
ProductID      0
Name           0
AlbumID        0
GenreID        0
Composer     977
dtype: int64
Unique Product IDs: True


#### Time Dimension

In chinook_data_mart, I've created and populated the Dim_Date table, similar to that in Lab02c.

In [65]:
mysql_args["dbname"] = "chinook_data_mart"
dim_date_df = get_sql_dataframe("SELECT date_key, full_date FROM dim_date", **mysql_args)
dim_date_df.head()

,date_key,full_date
0,20210101,2021-01-01
1,20210102,2021-01-02
2,20210103,2021-01-03
3,20210104,2021-01-04
4,20210105,2021-01-05


We will pair this with the Invoice_date to match in our final fact table. to_dateTime conversion happens there. 

#### Check to make everything is in line for fact table

In [68]:
# Checking Customer Dimension for unique CustomerID
customer_dim_check = """
SELECT CustomerID, COUNT(*) as cnt FROM customer_dim 
GROUP BY CustomerID HAVING cnt > 1;
"""
customer_dim_sample = get_sql_dataframe("SELECT * FROM customer_dim LIMIT 5", **mysql_args)
print("Customer Dimension Sample Data:")
customer_dim_sample.head()


Customer Dimension Sample Data:


,CustomerID,FirstName,LastName,Email,Phone,City,Country,PostalCode,SupportRepId
0,1,Luís,Gonçalves,luisg@embraer.com.br,+55 (12) 3923-5555,São José dos Campos,Brazil,12227-000,3
1,2,Leonie,Köhler,leonekohler@surfeu.de,+49 0711 2842222,Stuttgart,Germany,70174,5
2,3,François,Tremblay,ftremblay@gmail.com,+1 (514) 721-4711,Montréal,Canada,H2G 1A7,3
3,4,Bjørn,Hansen,bjorn.hansen@yahoo.no,+47 22 44 22 22,Oslo,Norway,0171,4
4,5,František,Wichterlová,frantisekw@jetbrains.com,+420 2 4172 5555,Prague,Czech Republic,14700,4


In [70]:
# Checking Product (Track) Dimension for unique ProductID
product_dim_check = """
SELECT ProductID, COUNT(*) as cnt FROM product_dim 
GROUP BY ProductID HAVING cnt > 1;
"""
product_dim_sample = get_sql_dataframe("SELECT * FROM product_dim LIMIT 5", **mysql_args)
print("Product Dimension Sample Data:")
product_dim_sample.head()

Product Dimension Sample Data:


,ProductID,ProductName,AlbumTitle,ArtistName,Genre,MediaType,Composer,Bytes,UnitPrice,Duration
0,1,For Those About To Rock (We Salute You),For Those About To Rock We Salute You,AC/DC,Rock,MPEG audio file,"Angus Young, Malcolm Young, Brian Johnson",11170334,0.99,05:43
1,2,Balls to the Wall,Balls to the Wall,Accept,Rock,Protected AAC audio file,"U. Dirkschneider, W. Hoffmann, H. Frank, P. Ba...",5510424,0.99,05:42
2,3,Fast As a Shark,Restless and Wild,Accept,Rock,Protected AAC audio file,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",3990994,0.99,03:50
3,4,Restless and Wild,Restless and Wild,Accept,Rock,Protected AAC audio file,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",4331779,0.99,04:12
4,5,Princess of the Dawn,Restless and Wild,Accept,Rock,Protected AAC audio file,Deaffy & R.A. Smith-Diesel,6290521,0.99,06:15


In [72]:
# Checking Invoice Dimension for unique InvoiceID
invoice_dim_check = """
SELECT InvoiceID, COUNT(*) as cnt FROM invoice_dim 
GROUP BY InvoiceID HAVING cnt > 1;
"""
invoice_dim_sample = get_sql_dataframe("SELECT * FROM invoice_dim LIMIT 5", **mysql_args)
print("Invoice Dimension Sample Data:")
invoice_dim_sample.head()

Invoice Dimension Sample Data:


,InvoiceID,CustomerID,Date,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,TotalAmount
0,1,2,2021-01-01,Theodor-Heuss-Straße 34,Stuttgart,,Germany,70174,1.98
1,2,4,2021-01-02,Ullevålsveien 14,Oslo,,Norway,0171,3.96
2,3,8,2021-01-03,Grétrystraat 63,Brussels,,Belgium,1000,5.94
3,4,14,2021-01-06,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2021-01-11,69 Salem Street,Boston,MA,USA,2113,13.86


In [74]:
mysql_args["dbname"] = "chinook"  # Switch to OLTP database

# Check uniqueness of ProductID in Track table
track_check = """
SELECT ProductID, COUNT(*) as cnt 
FROM Track 
GROUP BY ProductID 
HAVING cnt > 1;
"""
track_sample = get_sql_dataframe("SELECT * FROM Track LIMIT 5", **mysql_args)
print("Track Table Sample Data:")
track_sample.head()

Track Table Sample Data:


,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,"U. Dirkschneider, W. Hoffmann, H. Frank, P. Ba...",342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


In [76]:
invoice_line_check = """
SELECT InvoiceLineID, COUNT(*) as cnt 
FROM InvoiceLine 
GROUP BY InvoiceLineID 
HAVING cnt > 1;
"""
invoice_line_sample = get_sql_dataframe("SELECT * FROM InvoiceLine LIMIT 5", **mysql_args)
print("Invoice Line Table Sample Data:")
invoice_line_sample.head()

Invoice Line Table Sample Data:


,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity
0,1,1,2,0.99,1
1,2,1,4,0.99,1
2,3,2,6,0.99,1
3,4,2,8,0.99,1
4,5,2,10,0.99,1


In [78]:
# Set to OLTP (chinook) for Track data and check consistency with product_dim
track_ids = get_sql_dataframe("SELECT DISTINCT TrackID FROM Track", **mysql_args)

# Switch to Data Mart for Product Dimension
mysql_args["dbname"] = "chinook_data_mart"
product_ids = get_sql_dataframe("SELECT DISTINCT ProductID FROM product_dim", **mysql_args)

# Find Track IDs not in Product IDs (if any)
unmatched_ids = track_ids[~track_ids['TrackID'].isin(product_ids['ProductID'])]
print("Unmatched Track IDs:", unmatched_ids)


Unmatched Track IDs: Empty DataFrame
Columns: [TrackID]
Index: []


With an empty data frame, we know TrackID and ProductID represents a track or song IS in the database, so this is a match. 

#### Sales Fact Table

In MySQL, I've created the fact table using the keys from each dimension table. Now we can extract data using python and merge the sources to create a structured dataset for Sales_Fact.

The process will be: 
1. Merge InvoiceLine with Invoice: Adds InvoiceDate and CustomerID to each invoice line.
2. Merge with Customer Dimension: Links each line to customer information.
3. Merge with Product (Track): Adds product details using ProductID.
4. Merge with Date Dimension: Join the date data to link with a Date Dimension.

In [83]:
mysql_args["dbname"] = "chinook"
invoice_line_df = get_sql_dataframe("SELECT InvoiceLineID, InvoiceID, TrackID AS ProductID, UnitPrice AS Price, Quantity FROM InvoiceLine", **mysql_args)
track_df = get_sql_dataframe("SELECT TrackID AS ProductID, Name AS TrackName, AlbumID, GenreID FROM Track", **mysql_args)

In [85]:
mysql_args["dbname"] = "chinook_data_mart"

product_dim_df = get_sql_dataframe("SELECT ProductID, ProductName, AlbumTitle, ArtistName, Genre, MediaType, Composer, UnitPrice, Duration FROM product_dim", **mysql_args)
customer_dim_df = get_sql_dataframe("SELECT CustomerID, FirstName, LastName FROM customer_dim", **mysql_args)
invoice_dim_df = get_sql_dataframe("SELECT InvoiceID, CustomerID, TotalAmount, Date FROM invoice_dim", **mysql_args)
date_dim_df = get_sql_dataframe("SELECT date_key, full_date FROM dim_date", **mysql_args)

In [87]:
# Step 2: Merge InvoiceLine with Invoice Dimension
sales_fact_data = invoice_line_df.merge(invoice_dim_df, on='InvoiceID', how='inner')
print("After merging InvoiceLine with Invoice Dimension:")
sales_fact_data.head()

After merging InvoiceLine with Invoice Dimension:


,InvoiceLineID,InvoiceID,ProductID,Price,Quantity,CustomerID,TotalAmount,Date
0,1,1,2,0.99,1,2,1.98,2021-01-01
1,2,1,4,0.99,1,2,1.98,2021-01-01
2,3,2,6,0.99,1,4,3.96,2021-01-02
3,4,2,8,0.99,1,4,3.96,2021-01-02
4,5,2,10,0.99,1,4,3.96,2021-01-02


In [89]:
# Step 3: Merge with Customer Dimension on CustomerID
sales_fact_data = sales_fact_data.merge(customer_dim_df, on='CustomerID', how='inner')
print("After merging with Customer Dimension:")
sales_fact_data.head()

After merging with Customer Dimension:


,InvoiceLineID,InvoiceID,ProductID,Price,Quantity,CustomerID,TotalAmount,Date,FirstName,LastName
0,1,1,2,0.99,1,2,1.98,2021-01-01,Leonie,Köhler
1,2,1,4,0.99,1,2,1.98,2021-01-01,Leonie,Köhler
2,3,2,6,0.99,1,4,3.96,2021-01-02,Bjørn,Hansen
3,4,2,8,0.99,1,4,3.96,2021-01-02,Bjørn,Hansen
4,5,2,10,0.99,1,4,3.96,2021-01-02,Bjørn,Hansen


In [91]:
# Step 4: Merge with Product Dimension on ProductID
sales_fact_data = sales_fact_data.merge(product_dim_df, on='ProductID', how='inner')
print("After merging with Product Dimension:")
sales_fact_data.head()

After merging with Product Dimension:


,InvoiceLineID,InvoiceID,ProductID,Price,Quantity,CustomerID,TotalAmount,Date,FirstName,LastName,ProductName,AlbumTitle,ArtistName,Genre,MediaType,Composer,UnitPrice,Duration
0,1,1,2,0.99,1,2,1.98,2021-01-01,Leonie,Köhler,Balls to the Wall,Balls to the Wall,Accept,Rock,Protected AAC audio file,"U. Dirkschneider, W. Hoffmann, H. Frank, P. Ba...",0.99,05:42
1,2,1,4,0.99,1,2,1.98,2021-01-01,Leonie,Köhler,Restless and Wild,Restless and Wild,Accept,Rock,Protected AAC audio file,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",0.99,04:12
2,3,2,6,0.99,1,4,3.96,2021-01-02,Bjørn,Hansen,Put The Finger On You,For Those About To Rock We Salute You,AC/DC,Rock,MPEG audio file,"Angus Young, Malcolm Young, Brian Johnson",0.99,03:25
3,4,2,8,0.99,1,4,3.96,2021-01-02,Bjørn,Hansen,Inject The Venom,For Those About To Rock We Salute You,AC/DC,Rock,MPEG audio file,"Angus Young, Malcolm Young, Brian Johnson",0.99,03:30
4,5,2,10,0.99,1,4,3.96,2021-01-02,Bjørn,Hansen,Evil Walks,For Those About To Rock We Salute You,AC/DC,Rock,MPEG audio file,"Angus Young, Malcolm Young, Brian Johnson",0.99,04:23


In [93]:
# Step 5: Convert InvoiceDate to match Date Dimension and merge
date_dim_df['full_date'] = pd.to_datetime(date_dim_df['full_date'])
sales_fact_data = sales_fact_data.merge(date_dim_df, left_on='Date', right_on='full_date', how='left')
print("After merging with Date Dimension:")
sales_fact_data.head()

After merging with Date Dimension:


,InvoiceLineID,InvoiceID,ProductID,Price,Quantity,CustomerID,TotalAmount,Date,FirstName,LastName,ProductName,AlbumTitle,ArtistName,Genre,MediaType,Composer,UnitPrice,Duration,date_key,full_date
0,1,1,2,0.99,1,2,1.98,2021-01-01,Leonie,Köhler,Balls to the Wall,Balls to the Wall,Accept,Rock,Protected AAC audio file,"U. Dirkschneider, W. Hoffmann, H. Frank, P. Ba...",0.99,05:42,20210101,2021-01-01
1,2,1,4,0.99,1,2,1.98,2021-01-01,Leonie,Köhler,Restless and Wild,Restless and Wild,Accept,Rock,Protected AAC audio file,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",0.99,04:12,20210101,2021-01-01
2,3,2,6,0.99,1,4,3.96,2021-01-02,Bjørn,Hansen,Put The Finger On You,For Those About To Rock We Salute You,AC/DC,Rock,MPEG audio file,"Angus Young, Malcolm Young, Brian Johnson",0.99,03:25,20210102,2021-01-02
3,4,2,8,0.99,1,4,3.96,2021-01-02,Bjørn,Hansen,Inject The Venom,For Those About To Rock We Salute You,AC/DC,Rock,MPEG audio file,"Angus Young, Malcolm Young, Brian Johnson",0.99,03:30,20210102,2021-01-02
4,5,2,10,0.99,1,4,3.96,2021-01-02,Bjørn,Hansen,Evil Walks,For Those About To Rock We Salute You,AC/DC,Rock,MPEG audio file,"Angus Young, Malcolm Young, Brian Johnson",0.99,04:23,20210102,2021-01-02


In [95]:
column_order = [
    'InvoiceLineID', 'InvoiceID', 'ProductID', 'UnitPrice', 'Quantity', 'date_key', 
    'CustomerID',  
    ]

# Select and reorder columns to keep only those in column_order
sales_fact_data = sales_fact_data[column_order]

print("Final Sales Fact Table with Reordered Columns:")
sales_fact_data.head()

Final Sales Fact Table with Reordered Columns:


,InvoiceLineID,InvoiceID,ProductID,UnitPrice,Quantity,date_key,CustomerID
0,1,1,2,0.99,1,20210101,2
1,2,1,4,0.99,1,20210101,2
2,3,2,6,0.99,1,20210102,4
3,4,2,8,0.99,1,20210102,4
4,5,2,10,0.99,1,20210102,4


In [97]:
mysql_args["dbname"] = "chinook_data_mart"
db_url = f"mysql+pymysql://{mysql_args['uid']}:{mysql_args['pwd']}@{mysql_args['hostname']}/{mysql_args['dbname']}"
engine = create_engine(db_url)

In [99]:
sales_fact_data.to_sql('Sales_Fact', con=engine, if_exists='replace', index=False)

2240

In [101]:
df_sales_fact_sample = pd.read_sql_query("SELECT * FROM Sales_Fact LIMIT 5;", con=engine)
print("Sample data from Sales_Fact:")
df_sales_fact_sample

Sample data from Sales_Fact:


,InvoiceLineID,InvoiceID,ProductID,UnitPrice,Quantity,date_key,CustomerID
0,1,1,2,0.99,1,20210101,2
1,2,1,4,0.99,1,20210101,2
2,3,2,6,0.99,1,20210102,4
3,4,2,8,0.99,1,20210102,4
4,5,2,10,0.99,1,20210102,4


### Queries to Test the Sales Fact Table

#### This query identifies the top 10 highest-spending customers and their preferred genres, helping us understand their behavior.

In [105]:
sql_top_customers = """
SELECT 
    c.CustomerID,
    c.FirstName,
    c.LastName,
    SUM(f.UnitPrice * f.Quantity) AS TotalSpent,
    SUM(f.Quantity) AS TotalQuantity
FROM 
    Sales_Fact f
JOIN 
    customer_dim c ON f.CustomerID = c.CustomerID
GROUP BY 
    c.CustomerID, c.FirstName, c.LastName
ORDER BY 
    TotalSpent DESC
LIMIT 10;
"""

# Execute the query and load it into a DataFrame
df_top_customers = pd.read_sql_query(sql_top_customers, con=engine)

# Display the result
print("Top 10 Highest Spending Customers:")
df_top_customers

Top 10 Highest Spending Customers:


,CustomerID,FirstName,LastName,TotalSpent,TotalQuantity
0,6,Helena,Holý,49.62,38.0
1,26,Richard,Cunningham,47.62,38.0
2,57,Luis,Rojas,46.62,38.0
3,45,Ladislav,Kovács,45.62,38.0
4,46,Hugh,O'Reilly,45.62,38.0
5,37,Fynn,Zimmermann,43.62,38.0
6,24,Frank,Ralston,43.62,38.0
7,28,Julia,Barnett,43.62,38.0
8,25,Victor,Stevens,42.62,38.0
9,7,Astrid,Gruber,42.62,38.0


#### This query shows the top spenders' favorite Genres

In [108]:
sql_top_customers = """
SELECT 
    customer_data.CustomerID,
    customer_data.FirstName,
    customer_data.LastName,
    customer_data.TotalSpent,
    customer_data.TotalQuantity,
    favorite_genre.Genre AS FavoriteGenre
FROM 
    (SELECT 
        c.CustomerID,
        c.FirstName,
        c.LastName,
        SUM(f.UnitPrice * f.Quantity) AS TotalSpent,
        SUM(f.Quantity) AS TotalQuantity
    FROM 
        Sales_Fact f
    JOIN 
        customer_dim c ON f.CustomerID = c.CustomerID
    GROUP BY 
        c.CustomerID, c.FirstName, c.LastName
    ) AS customer_data
JOIN 
    (SELECT 
        f.CustomerID,
        p.Genre,
        SUM(f.Quantity) AS GenreQuantity
    FROM 
        Sales_Fact f
    JOIN 
        product_dim p ON f.ProductID = p.ProductID
    GROUP BY 
        f.CustomerID, p.Genre
    ) AS favorite_genre
ON 
    customer_data.CustomerID = favorite_genre.CustomerID
AND 
    favorite_genre.GenreQuantity = (
        SELECT MAX(GenreQuantity) 
        FROM (
            SELECT 
                f.CustomerID,
                p.Genre,
                SUM(f.Quantity) AS GenreQuantity
            FROM 
                Sales_Fact f
            JOIN 
                product_dim p ON f.ProductID = p.ProductID
            GROUP BY 
                f.CustomerID, p.Genre
        ) AS genre_subquery
        WHERE 
            genre_subquery.CustomerID = customer_data.CustomerID
    )
ORDER BY 
    customer_data.TotalSpent DESC
LIMIT 10;
"""

# Execute the query and load it into a DataFrame
df_top_customers = pd.read_sql_query(sql_top_customers, con=engine)

# Display the result
print("Top 10 Highest Spending Customers with Favorite Genre:")
df_top_customers

Top 10 Highest Spending Customers with Favorite Genre:


,CustomerID,FirstName,LastName,TotalSpent,TotalQuantity,FavoriteGenre
0,6,Helena,Holý,49.62,38.0,Rock
1,26,Richard,Cunningham,47.62,38.0,Rock
2,57,Luis,Rojas,46.62,38.0,Rock
3,46,Hugh,O'Reilly,45.62,38.0,Rock
4,45,Ladislav,Kovács,45.62,38.0,Rock
5,37,Fynn,Zimmermann,43.62,38.0,Rock
6,24,Frank,Ralston,43.62,38.0,Rock
7,28,Julia,Barnett,43.62,38.0,Rock
8,25,Victor,Stevens,42.62,38.0,Rock
9,7,Astrid,Gruber,42.62,38.0,Rock


To verify that "Rock" is indeed the favorite genre for each of the top 10 highest spenders, you can cross-check the total quantity of "Rock" purchases for these customers against other genres they’ve purchased.

In [111]:
# Define the modified SQL query to display the top two genres for each top customer
top_two_genres_query = """
WITH CustomerGenreRank AS (
    SELECT 
        f.CustomerID,
        c.FirstName,
        c.LastName,
        p.Genre,
        SUM(f.Quantity) AS TotalGenreQuantity,
        ROW_NUMBER() OVER (PARTITION BY f.CustomerID ORDER BY SUM(f.Quantity) DESC) AS GenreRank
    FROM 
        Sales_Fact f
    JOIN 
        customer_dim c ON f.CustomerID = c.CustomerID
    JOIN 
        product_dim p ON f.ProductID = p.ProductID
    WHERE 
        f.CustomerID IN (
            SELECT customer_data.CustomerID
            FROM (
                SELECT 
                    c.CustomerID,
                    SUM(f.UnitPrice * f.Quantity) AS TotalSpent
                FROM 
                    Sales_Fact f
                JOIN 
                    customer_dim c ON f.CustomerID = c.CustomerID
                GROUP BY 
                    c.CustomerID
                ORDER BY 
                    TotalSpent DESC
                LIMIT 10
            ) AS customer_data
        )
    GROUP BY 
        f.CustomerID, c.FirstName, c.LastName, p.Genre
)
SELECT 
    CustomerID,
    FirstName,
    LastName,
    Genre AS TopGenre,
    TotalGenreQuantity
FROM 
    CustomerGenreRank
WHERE 
    GenreRank <= 2
ORDER BY 
    CustomerID, GenreRank;
"""

# Execute the query and load it into a DataFrame
df_top_two_genres = pd.read_sql_query(top_two_genres_query, con=engine)

# Display the result
print("Top Two Genres for Each of the Top 10 Spending Customers:")
df_top_two_genres

Top Two Genres for Each of the Top 10 Spending Customers:


,CustomerID,FirstName,LastName,TopGenre,TotalGenreQuantity
0,6,Helena,Holý,Rock,10.0
1,6,Helena,Holý,Latin,6.0
2,7,Astrid,Gruber,Rock,15.0
3,7,Astrid,Gruber,Metal,7.0
4,24,Frank,Ralston,Rock,12.0
5,24,Frank,Ralston,Latin,5.0
6,25,Victor,Stevens,Rock,14.0
7,25,Victor,Stevens,Latin,7.0
8,26,Richard,Cunningham,Rock,14.0
9,26,Richard,Cunningham,Latin,7.0


#### This Query Finds the Most Bought Gengre for each city

In [130]:
most_bought_genre_by_city_query = """
WITH CityGenrePopularity AS (
    SELECT 
        c.City,
        p.Genre,
        COUNT(f.InvoiceLineID) AS PurchaseCount,
        ROW_NUMBER() OVER (PARTITION BY c.City ORDER BY COUNT(f.InvoiceLineID) DESC) AS GenreRank
    FROM 
        Sales_Fact f
    JOIN 
        customer_dim c ON f.CustomerID = c.CustomerID
    JOIN 
        product_dim p ON f.ProductID = p.ProductID
    GROUP BY 
        c.City, p.Genre
),
CityTotalPurchases AS (
    SELECT 
        c.City,
        COUNT(f.InvoiceLineID) AS TotalCityPurchases
    FROM 
        Sales_Fact f
    JOIN 
        customer_dim c ON f.CustomerID = c.CustomerID
    GROUP BY 
        c.City
)
SELECT 
    cg.City AS City,
    cg.Genre AS MostBoughtGenre,
    cg.PurchaseCount AS GenrePurchaseCount,
    ctp.TotalCityPurchases
FROM 
    CityGenrePopularity cg
JOIN 
    CityTotalPurchases ctp ON cg.City = ctp.City
WHERE 
    cg.GenreRank = 1
ORDER BY 
    ctp.TotalCityPurchases DESC, cg.City;
"""

# Execute the query and load it into a DataFrame
df_most_bought_genre_by_city = pd.read_sql_query(most_bought_genre_by_city_query, con=engine)

# Display the result
print("Most Bought Genre by City (with separate GenrePurchaseCount and TotalCityPurchases):")
df_most_bought_genre_by_city.head(10)


Most Bought Genre by City (with separate GenrePurchaseCount and TotalCityPurchases):


,City,MostBoughtGenre,GenrePurchaseCount,TotalCityPurchases
0,Berlin,Rock,34,76
1,London,Rock,26,76
2,Mountain View,Rock,21,76
3,Paris,Rock,30,76
4,Prague,Rock,25,76
5,São Paulo,Rock,40,76
6,Amsterdam,Rock,18,38
7,Bordeaux,Metal,14,38
8,Boston,Latin,12,38
9,Brasília,Rock,11,38


In [116]:
# Monthly Sales Trends for Hip Hop/Rap Query (Only Months with Sales)
sql_actual_monthly_hiphop_sales = """
SELECT 
    d.calendar_year AS year,
    d.month_of_year AS month,
    SUM(f.Quantity) AS TotalHipHopSales
FROM 
    Sales_Fact f
JOIN 
    product_dim p ON f.ProductID = p.ProductID
JOIN 
    dim_date d ON f.date_key = d.date_key
WHERE 
    p.Genre = 'Hip Hop/Rap'
GROUP BY 
    d.calendar_year, d.month_of_year
ORDER BY 
    d.calendar_year, d.month_of_year;
"""

# Execute the query and load it into a DataFrame
df_actual_monthly_hiphop_sales = pd.read_sql_query(sql_actual_monthly_hiphop_sales, con=engine)

# Display the result
print("Monthly Sales Trends for Hip Hop/Rap (Only Months with Sales):")
df_actual_monthly_hiphop_sales


Monthly Sales Trends for Hip Hop/Rap (Only Months with Sales):


,year,month,TotalHipHopSales
0,2021,10,2.0
1,2022,3,3.0
2,2023,1,1.0
3,2023,6,3.0
4,2024,5,2.0
5,2024,10,2.0
6,2025,8,1.0
7,2025,9,3.0
